In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [17]:
hts_18 = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2018.csv', encoding = 'cp949')
hts_19 = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2019.csv', encoding = 'cp949')
hts_20 = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2020.csv', encoding = 'cp949')
hts_21 = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_HTS_2021.csv', encoding = 'cp949')

player_18 = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2018.csv', encoding = 'cp949')
player_19 = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2019.csv', encoding = 'cp949')
player_20 = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2020.csv', encoding = 'cp949')
player_21 = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_선수_2021.csv', encoding = 'cp949')

hitter_18 = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2018.csv', encoding = 'cp949')
hitter_19 = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2019.csv', encoding = 'cp949')
hitter_20 = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2020.csv', encoding = 'cp949')
hitter_21 = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_타자 기본_2021.csv', encoding = 'cp949')

schedule_21 = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_경기일정_2021.csv', encoding = 'cp949')
team = pd.read_csv('./01_제공데이터/2021 빅콘테스트_데이터분석분야_챔피언리그_스포츠테크_팀.csv', encoding = 'cp949')

In [18]:
hitter_player_18 = pd.merge(hitter_18, player_18, how ='left', left_on = ['PCODE', 'GYEAR'], right_on = ['PCODE', 'GYEAR'])
hitter_player_19 = pd.merge(hitter_19, player_19, how ='left', left_on = ['PCODE', 'GYEAR'], right_on = ['PCODE', 'GYEAR'])
hitter_player_20 = pd.merge(hitter_20, player_20, how ='left', left_on = ['PCODE', 'GYEAR'], right_on = ['PCODE', 'GYEAR'])
hitter_player_21 = pd.merge(hitter_21, player_21, how ='left', left_on = ['PCODE', 'GYEAR'], right_on = ['PCODE', 'GYEAR'])

In [19]:
hitter_player_18

,GYEAR,PCODE,GAMENUM,PA,AB,BA,HIT,HR,TOTB,SLG,...,BB,KK,IB,HP,GD,NAME,T_ID,POSITION,AGE_VA,MONEY
0,2018,60100,70,169,152,0.243,37,3,63,0.414,...,12,36,0,4,5,백창수,HH,외,30,5000만원
1,2018,60184,6,10,10,0.200,2,0,3,0.300,...,0,3,0,0,0,김창혁,HH,포,27,2700만원
2,2018,60288,1,1,1,0.000,0,0,0,0.000,...,0,0,0,0,0,김응민,SS,포,26,3000만원
3,2018,60343,83,174,162,0.216,35,8,63,0.389,...,8,49,0,3,3,이해창,KT,포,31,7100만원
4,2018,60456,4,8,6,0.500,3,0,3,0.500,...,2,0,0,0,0,백상원,SS,내,30,6000만원
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,2018,79530,21,15,14,0.286,4,0,6,0.429,...,1,5,0,0,1,양종민,OB,내,27,2900만원
271,2018,79608,130,549,494,0.342,169,23,278,0.563,...,36,57,1,10,13,안치홍,HT,내,28,32000만원
272,2018,79705,58,102,85,0.259,22,4,39,0.459,...,11,26,0,3,3,김회성,HH,내,32,5800만원
273,2018,99606,88,205,183,0.295,54,4,77,0.421,...,16,49,1,5,5,정성훈,HT,내,38,10000만원


# 1. 30타석 이상 선수 필터링

In [20]:
player_list_18 = list(hitter_player_18[hitter_player_18.PA >= 30].NAME)
len(player_list_18)

195

In [21]:
player_list_19 = list(hitter_player_19[hitter_player_19.PA >= 30].NAME)
len(player_list_19)

208

In [22]:
player_list_20 = list(hitter_player_20[hitter_player_20.PA >= 30].NAME)
len(player_list_20)

210

In [23]:
player_list_21 = list(hitter_player_21[hitter_player_21.PA >= 30].NAME)
len(player_list_21)

171

# 2. 연도별 선수 정보

# 2.1) 2018년 선수정보

In [21]:
from selenium.common.exceptions import NoSuchElementException

result_18 = []
manuallycheck_18 = []

url = 'http://www.statiz.co.kr/player.php?name=%s&search='

driver = webdriver.Chrome(executable_path = 'chromedriver')
bydateselect = 'body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(1) > div > div.col-xs-12.col-sm-8.col-md-8.col-lg-12 > div > div.table-responsive.no-padding.box > table > tbody > tr > td > a:nth-child(4)'
closead = '#foin_close_02eu > a > img'

for i in tqdm(range(len(player_list_18))):
    player_path = url % player_list_18[i]
    driver.implicitly_wait(3)    
    driver.get(player_path)
    
    try:
        select = driver.find_element_by_css_selector(bydateselect)
        select.click()
        driver.implicitly_wait(3)

        select2 = driver.find_element_by_css_selector(closead)
        select2.click()
        driver.implicitly_wait(3)    

        select3 = driver.find_element_by_partial_link_text("2018")
        driver.implicitly_wait(3)
        select3.click()
        driver.implicitly_wait(3)

        table = driver.find_element_by_class_name('table.table-striped.table-responsive.table-condensed.no-space.table-bordered')
        tbody = table.find_element_by_tag_name("tbody")
        rows = tbody.find_elements_by_class_name("oddrow_stz0")+tbody.find_elements_by_class_name("evenrow_stz0")

        for index, value in enumerate(rows):
            result_18.append([player_list_18[i],value.text])
        time.sleep(3)
    
    except NoSuchElementException:
        manuallycheck_18.append(player_list_18[i])

           
df_result_18 = pd.DataFrame(result_18)



result_split_18= []
for i in df_result_18[1]:
    result_split_18.append(i.split(' '))

result_18_f = pd.concat([df_result_18[0],pd.DataFrame(result_split_18)], axis = 1)
result_18_f.columns = ['이름', '날짜', '상대', '결과', '점수', '타순', 'P', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타', 'OPS', '투구', 'avLI', 'RE24', 'WPA']
result_18_f

100%|██████████████████████████████████████████████████████████████████████████████| 195/195 [1:21:00<00:00, 24.92s/it]


,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,정훈,04-29,한화,W,4:3,7,3B,1,2,0,...,0,1,0.000,0.000,0.000,0.000,8,1.51,-0.72,-0.046
1,정훈,05-02,KIA,L,6:12,6,1B,1,4,0,...,0,0,0.125,0.111,0.250,0.361,12,1.09,-0.33,-0.032
2,정훈,05-04,@SK,L,4:5,9,PH,0,1,1,...,0,0,0.300,0.273,0.800,1.073,3,0.44,1.00,0.025
3,정훈,05-08,@LG,W,4:2,5,PH,0,1,0,...,0,0,0.333,0.308,0.750,1.058,6,0.49,-0.26,-0.013
4,정훈,05-10,@LG,W,7:2,9,PH,0,1,0,...,0,0,0.333,0.313,0.667,0.979,3,0.02,-0.26,-0.001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14640,김회성,10-03,롯데,W,7:6,9,3B,1,0,0,...,0,0,0.227,0.326,0.360,0.686,16,0.75,0.67,0.053
14641,김회성,10-09,@KT,W,10:6,7,3B,1,4,2,...,0,0,0.253,0.352,0.430,0.782,19,0.36,2.67,0.211
14642,김회성,10-13,NC,W,10:8,7,3B,1,2,3,...,0,0,0.259,0.364,0.459,0.822,18,0.74,2.01,0.247
14643,김회성,10-20,넥센,L,5:7,5,3B,0,2,0,...,0,0,0.250,0.250,0.250,0.500,6,1.31,-0.50,-0.064


In [22]:
len(set(result_18_f.이름))

160

In [23]:
result_18_f.isnull().sum()

이름      0
날짜      0
상대      0
결과      0
점수      0
타순      0
P       0
선발      0
타수      0
득점      0
안타      0
2타      0
3타      0
홈런      0
루타      0
타점      0
도루      0
도실      0
볼넷      0
사구      0
고4      0
삼진      0
병살      0
희타      0
희비      0
타율      0
출루      0
장타      0
OPS     0
투구      0
avLI    0
RE24    0
WPA     0
dtype: int64

In [24]:
len(manuallycheck_18)

35

In [25]:
result_18_f[result_18_f.투구.isnull()]

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA


In [26]:
manuallycheck_18

['백창수',
 '이해창',
 '이상호',
 '홍재호',
 '정진호',
 '이원재',
 '김재현',
 '윤수강',
 '한동민',
 '백민기',
 '조홍석',
 '박준태',
 '박광열',
 '김민혁',
 '김성훈',
 '최원준',
 '이동훈',
 '나종덕',
 '가르시아',
 '김태균',
 '윤석민',
 '박병호',
 '이원석',
 '이병규',
 '김현수',
 '김태완',
 '김성현',
 '이재원',
 '이명기',
 '김재현',
 '박건우',
 '김지수',
 '김상수',
 '정성훈',
 '이진영']

In [31]:
#백창수(백진우), 박광열(박대온)   검색 결과 없음

extra_18 = [
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%ED%95%B4%EC%B0%BD&birth=1987-05-11&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EC%83%81%ED%98%B8&birth=1989-02-05&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%ED%99%8D%EC%9E%AC%ED%98%B8&birth=1987-05-10&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%A0%95%EC%A7%84%ED%98%B8&birth=1988-10-02&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EC%9B%90%EC%9E%AC&birth=1989-05-20&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%9E%AC%ED%98%84&birth=1993-03-18&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%9E%AC%ED%98%84&birth=1987-11-07&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9C%A4%EC%97%AC%EC%9A%B4&birth=1990-02-22&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%ED%95%9C%EC%9C%A0%EC%84%AC&birth=1989-08-09&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%B1%EB%8F%99%ED%9B%88&birth=1990-09-16&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%A1%B0%ED%99%8D%EC%84%9D&birth=1990-02-23&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%95%EC%A4%80%ED%83%9C&birth=1991-07-26&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EB%AF%BC%ED%98%81&birth=1996-05-03&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%84%B1%ED%9B%88&birth=1993-10-27&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%B5%9C%EC%9B%90%EC%A4%80&birth=1997-03-23&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EB%8F%99%ED%9B%88&birth=1996-07-24&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EA%B0%80%EB%A5%B4%EC%8B%9C%EC%95%84&birth=1985-04-12', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%ED%83%9C%EA%B7%A0&birth=1982-05-29&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9C%A4%EC%84%9D%EB%AF%BC&birth=1985-09-04&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%95%EB%B3%91%ED%98%B8&birth=1986-07-10&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EC%9B%90%EC%84%9D&birth=1986-10-21&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EB%B3%91%EA%B7%9C&birth=1983-10-09&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%ED%98%84%EC%88%98&birth=1988-01-12&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EA%B9%80%ED%83%9C%EC%99%84&birth=1984-01-27', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%84%B1%ED%98%84&birth=1987-03-09&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EC%9E%AC%EC%9B%90&birth=1988-02-24&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EB%AA%85%EA%B8%B0&birth=1987-12-26&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%95%EA%B1%B4%EC%9A%B0&birth=1990-09-08&re=0&se=&da=&year=2018&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%A7%80%EC%88%98&birth=1986-08-23&re=0&se=&da=&year=2018&cv=',
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%83%81%EC%88%98&birth=1990-03-23&re=0&se=&da=&year=2018&cv=',
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%A0%95%EC%84%B1%ED%9B%88&birth=1980-06-27',
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%9D%B4%EC%A7%84%EC%98%81&birth=1980-06-15'] 

extra_18_name = ['이해창', '이상호', '홍재호','정진호','이원재','WO김재현','SK김재현','윤수강','한동민','백민기','조홍석',
                 '박준태','김민혁','김성훈','최원준','이동훈', '가르시아','김태균','윤석민','박병호',
                '이원석','이병규','김현수','김태완','김성현','이재원','이명기','박건우','김지수','김상수','정성훈','이진영']

In [32]:
len(extra_18), len(extra_18_name)

(32, 32)

In [33]:
hitter_player_18[hitter_player_18.NAME == '이진영']

,GYEAR,PCODE,GAMENUM,PA,AB,BA,HIT,HR,TOTB,SLG,...,BB,KK,IB,HP,GD,NAME,T_ID,POSITION,AGE_VA,MONEY
274,2018,99810,110,312,283,0.318,90,3,118,0.417,...,25,35,1,0,3,이진영,KT,외,38,40000만원


In [34]:
extra_result_18 = []

driver = webdriver.Chrome(executable_path = 'chromedriver')
bydateselect = 'body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(1) > div > div.col-xs-12.col-sm-8.col-md-8.col-lg-12 > div > div.table-responsive.no-padding.box > table > tbody > tr > td > a:nth-child(4)'
closead = '#foin_close_02eu > a > img'

for i in tqdm(range(len(extra_18))):
    player_path = i
    driver.implicitly_wait(3)    
    driver.get(extra_18[i])
    
    table = driver.find_element_by_class_name('table.table-striped.table-responsive.table-condensed.no-space.table-bordered')
    tbody = table.find_element_by_tag_name("tbody")
    rows = tbody.find_elements_by_class_name("oddrow_stz0")+tbody.find_elements_by_class_name("evenrow_stz0")

    for index, value in enumerate(rows):
        extra_result_18.append([extra_18_name[i],value.text])
    time.sleep(3)
    
           
df_extra_18 = pd.DataFrame(extra_result_18)


extra_result_18_split= []
for i in df_extra_18[1]:
    extra_result_18_split.append(i.split(' '))

extra_result_18_f = pd.concat([df_extra_18[0],pd.DataFrame(extra_result_18_split)], axis = 1)
extra_result_18_f.columns = ['이름', '날짜', '상대', '결과', '점수', '타순', 'P', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타', 'OPS', '투구', 'avLI', 'RE24', 'WPA']
extra_result_18_f

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [17:18<00:00, 32.45s/it]


,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,이해창,03-25,@KIA,L,1:14,9,C,0,1,0,...,0,0,0.000,0.000,0.000,0.000,3,0.01,-0.32,0.000
1,이해창,03-28,@SK,W,8:5,8,C,0,0,0,...,0,0,0.000,0.000,0.000,0.000,0,0.00,0.00,0.000
2,이해창,04-01,두산,W,9:4,8,C,1,4,1,...,0,0,0.444,0.444,1.111,1.556,15,0.97,1.63,0.217
3,이해창,04-04,@넥센,L,2:10,8,C,1,2,1,...,0,0,0.467,0.467,1.133,1.600,8,0.42,0.88,0.063
4,이해창,04-08,한화,L,8:12,7,C,1,3,0,...,0,0,0.318,0.318,0.773,1.091,20,0.47,-1.23,-0.044
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,이진영,09-29,롯데,L,3:8,5,DH,1,4,0,...,0,0,0.307,0.366,0.394,0.760,7,0.64,0.01,-0.066
2896,이진영,10-02,@LG,L,6:10,2,DH,1,5,2,...,0,0,0.310,0.367,0.406,0.773,21,1.15,1.45,0.146
2897,이진영,10-06,삼성,W,6:1,2,DH,1,4,0,...,0,0,0.305,0.360,0.398,0.758,20,0.90,0.07,-0.049
2898,이진영,10-10,@롯데,W,10:1,2,DH,1,4,0,...,0,0,0.310,0.364,0.405,0.769,13,0.56,0.70,-0.020


In [35]:
result_18_ff = pd.concat([result_18_f,extra_result_18_f])
result_18_ff

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,정훈,04-29,한화,W,4:3,7,3B,1,2,0,...,0,1,0.000,0.000,0.000,0.000,8,1.51,-0.72,-0.046
1,정훈,05-02,KIA,L,6:12,6,1B,1,4,0,...,0,0,0.125,0.111,0.250,0.361,12,1.09,-0.33,-0.032
2,정훈,05-04,@SK,L,4:5,9,PH,0,1,1,...,0,0,0.300,0.273,0.800,1.073,3,0.44,1.00,0.025
3,정훈,05-08,@LG,W,4:2,5,PH,0,1,0,...,0,0,0.333,0.308,0.750,1.058,6,0.49,-0.26,-0.013
4,정훈,05-10,@LG,W,7:2,9,PH,0,1,0,...,0,0,0.333,0.313,0.667,0.979,3,0.02,-0.26,-0.001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2895,이진영,09-29,롯데,L,3:8,5,DH,1,4,0,...,0,0,0.307,0.366,0.394,0.760,7,0.64,0.01,-0.066
2896,이진영,10-02,@LG,L,6:10,2,DH,1,5,2,...,0,0,0.310,0.367,0.406,0.773,21,1.15,1.45,0.146
2897,이진영,10-06,삼성,W,6:1,2,DH,1,4,0,...,0,0,0.305,0.360,0.398,0.758,20,0.90,0.07,-0.049
2898,이진영,10-10,@롯데,W,10:1,2,DH,1,4,0,...,0,0,0.310,0.364,0.405,0.769,13,0.56,0.70,-0.020


In [36]:
result_18_ff[result_18_ff.장타.isnull()]

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA


# 2.2) 2019년 선수정보

In [39]:
result_19 = []
manuallycheck_19 = []

url = 'http://www.statiz.co.kr/player.php?name=%s&search='

driver = webdriver.Chrome(executable_path = 'chromedriver')
bydateselect = 'body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(1) > div > div.col-xs-12.col-sm-8.col-md-8.col-lg-12 > div > div.table-responsive.no-padding.box > table > tbody > tr > td > a:nth-child(4)'
closead = '#foin_close_02eu > a > img'

for i in tqdm(range(len(player_list_19))):
    player_path = url % player_list_19[i]
    driver.implicitly_wait(3)    
    driver.get(player_path)
    
    try:
        select = driver.find_element_by_css_selector(bydateselect)
        select.click()
        driver.implicitly_wait(3)

        select2 = driver.find_element_by_css_selector(closead)
        select2.click()
        driver.implicitly_wait(3)    

        select3 = driver.find_element_by_partial_link_text("2019")
        driver.implicitly_wait(3)
        select3.click()
        driver.implicitly_wait(3)

        table = driver.find_element_by_class_name('table.table-striped.table-responsive.table-condensed.no-space.table-bordered')
        tbody = table.find_element_by_tag_name("tbody")
        rows = tbody.find_elements_by_class_name("oddrow_stz0")+tbody.find_elements_by_class_name("evenrow_stz0")

        for index, value in enumerate(rows):
            result_19.append([player_list_19[i],value.text])
        time.sleep(3)
    
    except NoSuchElementException:
        manuallycheck_19.append(player_list_19[i])

           
df_result_19 = pd.DataFrame(result_19)


result_split_19= []
for i in df_result_19[1]:
    result_split_19.append(i.split(' '))

result_19_f = pd.concat([df_result_19[0],pd.DataFrame(result_split_19)], axis = 1)
result_19_f.columns = ['이름', '날짜', '상대', '결과', '점수', '타순', 'P', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타', 'OPS', '투구', 'avLI', 'RE24', 'WPA']
result_19_f

100%|████████████████████████████████████████████████████████████████████████████████| 208/208 [36:45<00:00, 10.60s/it]


,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,정훈,03-23,키움,L,4:7,9,PH,0,1,0,...,0,0,0.000,0.000,0.000,0.000,3,0.50,-0.18,-0.013
1,정훈,03-27,삼성,L,4:23,2,RF,0,1,0,...,0,0,0.000,0.000,0.000,0.000,3,0.00,-0.46,0.000
2,정훈,03-29,@LG,L,1:2,6,PH,0,1,0,...,0,0,0.000,0.000,0.000,0.000,4,6.11,-0.51,-0.140
3,정훈,03-31,@LG,L,5:6,4,PR,0,0,0,...,0,0,0.000,0.000,0.000,0.000,0,0.00,0.00,0.000
4,정훈,04-04,@SK,L,6:7,5,CF,1,4,0,...,0,0,0.125,0.222,0.125,0.347,24,1.11,0.13,-0.050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14218,김회성,09-14,롯데,W,5:2,7,3B,1,3,1,...,0,1,0.245,0.301,0.383,0.684,16,1.71,-1.33,-0.059
14219,김회성,09-17,키움,W,1:0,3,1B,0,1,0,...,0,0,0.242,0.296,0.374,0.670,3,0.80,-0.34,-0.023
14220,김회성,09-23,@LG,W,9:1,1,1B,0,0,0,...,0,0,0.242,0.303,0.374,0.676,0,0.00,0.00,0.000
14221,김회성,09-26,@NC,W,4:2,7,1B,1,1,0,...,0,0,0.240,0.313,0.370,0.683,17,1.06,0.26,0.026


In [40]:
len(set(result_19_f.이름))

176

In [41]:
result_19_f.isnull().sum()

이름      0
날짜      0
상대      0
결과      0
점수      0
타순      0
P       0
선발      1
타수      1
득점      1
안타      1
2타      1
3타      1
홈런      1
루타      1
타점      1
도루      1
도실      1
볼넷      1
사구      1
고4      1
삼진      1
병살      1
희타      1
희비      1
타율      1
출루      1
장타      1
OPS     1
투구      1
avLI    1
RE24    1
WPA     1
dtype: int64

In [42]:
len(manuallycheck_19)

32

In [43]:
result_19_f[result_19_f.투구.isnull()]

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
4756,백승현,데이터가,없습니다.,검색,조건을,다시,확인하세요.,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [44]:
manuallycheck_19

['백창수',
 '이해창',
 '이상호',
 '정진호',
 '이원재',
 '한동민',
 '조홍석',
 '고장혁',
 '김민혁',
 '박찬호',
 '박준태',
 '김민수',
 '조용호',
 '김성훈',
 '최원준',
 '이동훈',
 '김민수',
 '나종덕',
 '페르난데스',
 '윌슨',
 '김태균',
 '윤석민',
 '박병호',
 '이원석',
 '김현수',
 '김성현',
 '이재원',
 '이명기',
 '김재현',
 '박건우',
 '김지수',
 '김상수']

In [45]:
hitter_player_19[hitter_player_19.NAME == '백창수']

,GYEAR,PCODE,GAMENUM,PA,AB,BA,HIT,HR,TOTB,SLG,...,BB,KK,IB,HP,GD,NAME,T_ID,POSITION,AGE_VA,MONEY
0,2019,60100,16,48,44,0.227,10,0,13,0.295,...,0,18,0,2,1,백창수,HH,외,31,6000만원


In [46]:
#백창수(백진우), 고장혁, 나종덕 검색 결과 없음

extra_19 = [
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%ED%95%B4%EC%B0%BD&birth=1987-05-11&re=0&se=&da=&year=2019&cv=',
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EC%83%81%ED%98%B8&birth=1989-02-05&re=0&se=&da=&year=2019&cv=',
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%A0%95%EC%A7%84%ED%98%B8&birth=1988-10-02&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EC%9B%90%EC%9E%AC&birth=1989-05-20&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%A1%B0%ED%99%8D%EC%84%9D&birth=1990-02-23&re=0&se=&da=&year=2019&cv=',
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EB%AF%BC%ED%98%81&birth=1995-11-21&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%95%EC%B0%AC%ED%98%B8&birth=1995-06-05&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%95%EC%A4%80%ED%83%9C&birth=1991-07-26&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EB%AF%BC%EC%88%98&birth=1991-03-02&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EB%AF%BC%EC%88%98&birth=1998-03-18&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%A1%B0%EC%9A%A9%ED%98%B8&birth=1989-09-09&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%84%B1%ED%9B%88&birth=1993-10-27&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%B5%9C%EC%9B%90%EC%A4%80&birth=1997-03-23&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EB%8F%99%ED%9B%88&birth=1996-07-24&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%ED%8E%98%EB%A5%B4%EB%82%9C%EB%8D%B0%EC%8A%A4&birth=1988-04-27&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%9C%8C%EC%8A%A8&birth=1990-07-29', #윌슨
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%ED%83%9C%EA%B7%A0&birth=1982-05-29&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9C%A4%EC%84%9D%EB%AF%BC&birth=1985-09-04&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%95%EB%B3%91%ED%98%B8&birth=1986-07-10&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EC%9B%90%EC%84%9D&birth=1986-10-21&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%ED%98%84%EC%88%98&birth=1988-01-12&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%84%B1%ED%98%84&birth=1987-03-09&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EC%9E%AC%EC%9B%90&birth=1988-02-24&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EB%AA%85%EA%B8%B0&birth=1987-12-26&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%9E%AC%ED%98%84&birth=1987-11-07&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%95%EA%B1%B4%EC%9A%B0&birth=1990-09-08&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%A7%80%EC%88%98&birth=1986-08-23&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%83%81%EC%88%98&birth=1990-03-23&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%B1%EC%8A%B9%ED%98%84&birth=1995-05-26&re=0&se=&da=&year=2019&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%ED%95%9C%EC%9C%A0%EC%84%AC&birth=1989-08-09&re=0&se=&da=&year=2019&cv='] 

extra_19_name = ['이해창','이상호','정진호','이원재','조홍석','김민혁','박찬호','박준태','SS김민수','LT김민수','조용호','김성훈',
                 '최원준','이동훈','페르난데스','윌슨','김태균','윤석민','박병호','이원석','김현수','김성현','이재원','이명기',
                 '김재현','박건우','김지수','김상수','백승현', '한동민']

In [47]:
len(extra_19), len(extra_19_name)

(30, 30)

In [48]:
extra_result_19 = []


driver = webdriver.Chrome(executable_path = 'chromedriver')
bydateselect = 'body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(1) > div > div.col-xs-12.col-sm-8.col-md-8.col-lg-12 > div > div.table-responsive.no-padding.box > table > tbody > tr > td > a:nth-child(4)'
closead = '#foin_close_02eu > a > img'

for i in tqdm(range(len(extra_19))):
    player_path = i
    driver.implicitly_wait(3)    
    driver.get(extra_19[i])
    
    table = driver.find_element_by_class_name('table.table-striped.table-responsive.table-condensed.no-space.table-bordered')
    tbody = table.find_element_by_tag_name("tbody")
    rows = tbody.find_elements_by_class_name("oddrow_stz0")+tbody.find_elements_by_class_name("evenrow_stz0")

    for index, value in enumerate(rows):
        extra_result_19.append([extra_19_name[i],value.text])
    time.sleep(3)
    
           
df_extra_19 = pd.DataFrame(extra_result_19)


extra_result_19_split= []
for i in df_extra_19[1]:
    extra_result_19_split.append(i.split(' '))

extra_result_19_f = pd.concat([df_extra_19[0],pd.DataFrame(extra_result_19_split)], axis = 1)
extra_result_19_f.columns = ['이름', '날짜', '상대', '결과', '점수', '타순', 'P', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타', 'OPS', '투구', 'avLI', 'RE24', 'WPA']
extra_result_19_f

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [03:51<00:00,  7.72s/it]


,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,이해창,03-23,@SK,L,4:7,9,PH,0,1,0,...,0,0,0.000,0.000,0.000,0.000,6,0.75,-0.26,-0.019
1,이해창,03-26,@NC,L,8:9,9,PH,0,1,0,...,0,0,0.000,0.000,0.000,0.000,7,2.70,-0.47,-0.070
2,이해창,03-31,KIA,L,2:4,8,C,1,3,0,...,0,0,0.000,0.000,0.000,0.000,13,1.14,-0.77,-0.091
3,이해창,04-05,LG,L,0:13,7,C,0,1,0,...,0,0,0.000,0.000,0.000,0.000,5,0.08,-0.26,-0.002
4,이해창,04-09,@키움,L,3:7,8,C,0,1,0,...,0,0,0.000,0.167,0.000,0.167,6,0.34,-0.26,-0.009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,한동민,09-19,두산,L,3:7,2,RF,1,4,0,...,0,0,0.269,0.379,0.404,0.783,18,0.66,-0.80,-0.066
2712,한동민,09-24,@KT,L,3:7,2,RF,1,3,0,...,0,0,0.269,0.380,0.402,0.782,17,0.70,0.08,-0.001
2713,한동민,09-28,@삼성,L,7:9,8,PH,0,1,0,...,0,0,0.265,0.375,0.397,0.772,6,2.87,-0.26,-0.073
2714,한동민,10-14,키움,L,0:3,5,RF,1,4,0,...,0,0,0.000,0.200,0.000,0.200,21,1.60,-0.75,-0.100


In [51]:
result_19_ff = pd.concat([result_19_f,extra_result_19_f])
result_19_ff

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,정훈,03-23,키움,L,4:7,9,PH,0,1,0,...,0,0,0.000,0.000,0.000,0.000,3,0.50,-0.18,-0.013
1,정훈,03-27,삼성,L,4:23,2,RF,0,1,0,...,0,0,0.000,0.000,0.000,0.000,3,0.00,-0.46,0.000
2,정훈,03-29,@LG,L,1:2,6,PH,0,1,0,...,0,0,0.000,0.000,0.000,0.000,4,6.11,-0.51,-0.140
3,정훈,03-31,@LG,L,5:6,4,PR,0,0,0,...,0,0,0.000,0.000,0.000,0.000,0,0.00,0.00,0.000
4,정훈,04-04,@SK,L,6:7,5,CF,1,4,0,...,0,0,0.125,0.222,0.125,0.347,24,1.11,0.13,-0.050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,한동민,09-19,두산,L,3:7,2,RF,1,4,0,...,0,0,0.269,0.379,0.404,0.783,18,0.66,-0.80,-0.066
2712,한동민,09-24,@KT,L,3:7,2,RF,1,3,0,...,0,0,0.269,0.380,0.402,0.782,17,0.70,0.08,-0.001
2713,한동민,09-28,@삼성,L,7:9,8,PH,0,1,0,...,0,0,0.265,0.375,0.397,0.772,6,2.87,-0.26,-0.073
2714,한동민,10-14,키움,L,0:3,5,RF,1,4,0,...,0,0,0.000,0.200,0.000,0.200,21,1.60,-0.75,-0.100


In [52]:
result_19_ff = result_19_ff.dropna(axis = 0)
result_19_ff

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,정훈,03-23,키움,L,4:7,9,PH,0,1,0,...,0,0,0.000,0.000,0.000,0.000,3,0.50,-0.18,-0.013
1,정훈,03-27,삼성,L,4:23,2,RF,0,1,0,...,0,0,0.000,0.000,0.000,0.000,3,0.00,-0.46,0.000
2,정훈,03-29,@LG,L,1:2,6,PH,0,1,0,...,0,0,0.000,0.000,0.000,0.000,4,6.11,-0.51,-0.140
3,정훈,03-31,@LG,L,5:6,4,PR,0,0,0,...,0,0,0.000,0.000,0.000,0.000,0,0.00,0.00,0.000
4,정훈,04-04,@SK,L,6:7,5,CF,1,4,0,...,0,0,0.125,0.222,0.125,0.347,24,1.11,0.13,-0.050
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2711,한동민,09-19,두산,L,3:7,2,RF,1,4,0,...,0,0,0.269,0.379,0.404,0.783,18,0.66,-0.80,-0.066
2712,한동민,09-24,@KT,L,3:7,2,RF,1,3,0,...,0,0,0.269,0.380,0.402,0.782,17,0.70,0.08,-0.001
2713,한동민,09-28,@삼성,L,7:9,8,PH,0,1,0,...,0,0,0.265,0.375,0.397,0.772,6,2.87,-0.26,-0.073
2714,한동민,10-14,키움,L,0:3,5,RF,1,4,0,...,0,0,0.000,0.200,0.000,0.200,21,1.60,-0.75,-0.100


# 2.3) 2020년 선수정보

In [53]:
result_20 = []
manuallycheck_20 = []

url = 'http://www.statiz.co.kr/player.php?name=%s&search='

driver = webdriver.Chrome(executable_path = 'chromedriver')
bydateselect = 'body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(1) > div > div.col-xs-12.col-sm-8.col-md-8.col-lg-12 > div > div.table-responsive.no-padding.box > table > tbody > tr > td > a:nth-child(4)'
closead = '#foin_close_02eu > a > img'

for i in tqdm(range(len(player_list_20))):
    player_path = url % player_list_20[i]
    driver.implicitly_wait(3)    
    driver.get(player_path)
    
    try:
        select = driver.find_element_by_css_selector(bydateselect)
        select.click()
        driver.implicitly_wait(3)

        select2 = driver.find_element_by_css_selector(closead)
        select2.click()
        driver.implicitly_wait(3)    

        select3 = driver.find_element_by_partial_link_text("2020")
        driver.implicitly_wait(3)
        select3.click()
        driver.implicitly_wait(3)

        table = driver.find_element_by_class_name('table.table-striped.table-responsive.table-condensed.no-space.table-bordered')
        tbody = table.find_element_by_tag_name("tbody")
        rows = tbody.find_elements_by_class_name("oddrow_stz0")+tbody.find_elements_by_class_name("evenrow_stz0")

        for index, value in enumerate(rows):
            result_20.append([player_list_20[i],value.text])
        time.sleep(3)
    
    except NoSuchElementException:
        manuallycheck_20.append(player_list_20[i])

           
df_result_20 = pd.DataFrame(result_20)


result_split_20= []
for i in df_result_20[1]:
    result_split_20.append(i.split(' '))

result_20_f = pd.concat([df_result_20[0],pd.DataFrame(result_split_20)], axis = 1)
result_20_f.columns = ['이름', '날짜', '상대', '결과', '점수', '타순', 'P', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타', 'OPS', '투구', 'avLI', 'RE24', 'WPA']
result_20_f

100%|████████████████████████████████████████████████████████████████████████████████| 210/210 [43:04<00:00, 12.31s/it]


,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,강현우,05-06,롯데,L,4:9,8,C,0,1,0,...,0,0,0.000,0.000,0.000,0.000,1,0.31,-0.12,-0.008
1,강현우,05-08,@두산,W,12:3,8,PR,0,0,0,...,0,0,0.000,0.000,0.000,0.000,0,0.00,0.00,0.000
2,강현우,05-13,@NC,L,4:5,8,C,0,0,0,...,0,0,0.500,0.500,1.000,1.500,0,0.00,0.00,0.000
3,강현우,05-16,삼성,W,10:4,2,C,0,0,0,...,0,0,0.500,0.500,1.000,1.500,0,0.00,0.00,0.000
4,강현우,05-21,한화,L,4:9,8,C,0,1,0,...,0,0,0.429,0.429,0.714,1.143,5,0.62,-0.46,-0.017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15507,안치홍,10-21,@SK,L,3:11,7,PH,0,2,0,...,0,0,0.283,0.348,0.398,0.746,6,0.02,0.80,0.002
15508,안치홍,10-23,@SK,W,3:0,6,2B,1,3,0,...,0,0,0.291,0.355,0.414,0.769,11,0.70,0.58,0.043
15509,안치홍,10-25,@KT,L,5:10,4,PH,0,2,0,...,0,0,0.286,0.350,0.407,0.758,2,0.34,-0.93,-0.018
15510,안치홍,10-28,NC,L,1:7,7,2B,1,2,0,...,0,0,0.285,0.349,0.405,0.754,4,0.72,-0.51,-0.039


In [54]:
len(set(result_20_f.이름))

181

In [55]:
len(manuallycheck_20)

29

In [56]:
result_20_f[result_20_f.장타.isnull()]

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
6212,백승현,데이터가,없습니다.,검색,조건을,다시,확인하세요.,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [57]:
manuallycheck_20

['천성호',
 '박정현',
 '화이트',
 '이해창',
 '이상호',
 '정진호',
 '이원재',
 '한동민',
 '김민혁',
 '김경호',
 '박찬호',
 '박준태',
 '조용호',
 '최원준',
 '이동훈',
 '박준영',
 '페르난데스',
 '김주형',
 '김태균',
 '윤석민',
 '박병호',
 '이원석',
 '이병규',
 '김현수',
 '김성현',
 '이재원',
 '이명기',
 '박건우',
 '김상수']

In [85]:
hitter_player_20[hitter_player_20.NAME == '이재원']

,GYEAR,PCODE,GAMENUM,PA,AB,BA,HIT,HR,TOTB,SLG,...,BB,KK,IB,HP,GD,NAME,T_ID,POSITION,AGE_VA,MONEY
180,2020,68106,16,22,20,0.050,1,0,1,0.050,...,2,11,0,0,0,이재원,LG,외,20,2700만원
250,2020,76812,80,251,222,0.185,41,2,55,0.248,...,19,24,3,6,12,이재원,SK,포,32,130000만원


In [59]:
extra_20 = [
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%B2%9C%EC%84%B1%ED%98%B8&birth=1997-10-30&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%95%EC%A0%95%ED%98%84&birth=2001-07-27&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&name=%ED%99%94%EC%9D%B4%ED%8A%B8&birth=1990-10-29', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%ED%95%B4%EC%B0%BD&birth=1987-05-11&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EC%83%81%ED%98%B8&birth=1989-02-05&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%A0%95%EC%A7%84%ED%98%B8&birth=1988-10-02&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EC%9B%90%EC%9E%AC&birth=1989-05-20&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%ED%95%9C%EC%9C%A0%EC%84%AC&birth=1989-08-09&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EB%AF%BC%ED%98%81&birth=1995-11-21&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EA%B9%80%EA%B2%BD%ED%98%B8&birth=1995-07-31', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%95%EC%B0%AC%ED%98%B8&birth=1995-06-05&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%95%EC%A4%80%ED%83%9C&birth=1991-07-26&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%A1%B0%EC%9A%A9%ED%98%B8&birth=1989-09-09&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%B5%9C%EC%9B%90%EC%A4%80&birth=1997-03-23&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EB%8F%99%ED%9B%88&birth=1996-07-24&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%95%EC%A4%80%EC%98%81&birth=1997-08-05&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%ED%8E%98%EB%A5%B4%EB%82%9C%EB%8D%B0%EC%8A%A4&birth=1988-04-27&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%A3%BC%ED%98%95&birth=1996-03-05&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%ED%83%9C%EA%B7%A0&birth=1982-05-29&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9C%A4%EC%84%9D%EB%AF%BC&birth=1985-09-04&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%95%EB%B3%91%ED%98%B8&birth=1986-07-10&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EC%9B%90%EC%84%9D&birth=1986-10-21&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EB%B3%91%EA%B7%9C&birth=1983-10-09&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%ED%98%84%EC%88%98&birth=1988-01-12&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%84%B1%ED%98%84&birth=1987-03-09&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EC%9E%AC%EC%9B%90&birth=1988-02-24&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EC%9D%B4%EB%AA%85%EA%B8%B0&birth=1987-12-26&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%95%EA%B1%B4%EC%9A%B0&birth=1990-09-08&re=0&se=&da=&year=2020&cv=', 
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EA%B9%80%EC%83%81%EC%88%98&birth=1990-03-23&re=0&se=&da=&year=2020&cv=',
'http://www.statiz.co.kr/player.php?opt=3&sopt=0&name=%EB%B0%B1%EC%8A%B9%ED%98%84&birth=1995-05-26&re=0&se=&da=&year=2020&cv='] 

extra_20_name = ['천성호','박정현','화이트','이해창','이상호','정진호','이원재','한동민','김민혁','김경호','박찬호','박준태','조용호',
                 '최원준','이동훈','박준영','페르난데스','김주형','김태균','윤석민','박병호','이원석','이병규','김현수','김성현',
                 '이재원','이명기','박건우','김상수', '백승현']

In [60]:
len(extra_20), len(extra_20_name)

(30, 30)

In [61]:
extra_result_20 = []


driver = webdriver.Chrome(executable_path = 'chromedriver')
bydateselect = 'body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(1) > div > div.col-xs-12.col-sm-8.col-md-8.col-lg-12 > div > div.table-responsive.no-padding.box > table > tbody > tr > td > a:nth-child(4)'
closead = '#foin_close_02eu > a > img'

for i in tqdm(range(len(extra_20))):
    player_path = i
    driver.implicitly_wait(3)    
    driver.get(extra_20[i])
    
    table = driver.find_element_by_class_name('table.table-striped.table-responsive.table-condensed.no-space.table-bordered')
    tbody = table.find_element_by_tag_name("tbody")
    rows = tbody.find_elements_by_class_name("oddrow_stz0")+tbody.find_elements_by_class_name("evenrow_stz0")

    for index, value in enumerate(rows):
        extra_result_20.append([extra_20_name[i],value.text])
    time.sleep(3)
    
           
df_extra_20 = pd.DataFrame(extra_result_20)


extra_result_20_split= []
for i in df_extra_20[1]:
    extra_result_20_split.append(i.split(' '))

extra_result_20_f = pd.concat([df_extra_20[0],pd.DataFrame(extra_result_20_split)], axis = 1)
extra_result_20_f.columns = ['이름', '날짜', '상대', '결과', '점수', '타순', 'P', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타', 'OPS', '투구', 'avLI', 'RE24', 'WPA']
extra_result_20_f

100%|██████████████████████████████████████████████████████████████████████████████████| 30/30 [04:26<00:00,  8.89s/it]


,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,천성호,06-03,두산,W,7:2,9,DH,1,3,0,...,0,0,0.667,0.667,1.333,2.000,12,0.30,0.20,0.000
1,천성호,06-06,@롯데,L,0:1,8,PR,0,0,0,...,0,0,0.333,0.429,0.667,1.095,0,0.00,0.00,0.000
2,천성호,06-10,KIA,L,0:10,8,3B,1,2,0,...,0,0,0.364,0.417,0.545,0.962,4,0.05,-0.05,-0.001
3,천성호,06-13,@삼성,L,3:5,8,3B,1,2,0,...,0,0,0.471,0.550,0.647,1.197,12,0.78,0.37,0.047
4,천성호,06-14,@삼성,L,0:12,7,3B,1,3,0,...,0,0,0.364,0.462,0.500,0.962,21,0.29,-0.23,-0.018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2618,백승현,07-07,@두산,L,6:9,8,3B,1,3,2,...,0,0,0.350,0.409,0.400,0.809,10,0.71,-0.05,-0.019
2619,백승현,07-09,@두산,L,0:6,8,3B,1,2,0,...,0,0,0.280,0.357,0.320,0.677,9,1.11,-0.52,-0.057
2620,백승현,07-11,NC,D,6:6,9,3B,1,2,0,...,2,0,0.250,0.344,0.286,0.629,15,0.92,-0.71,-0.037
2621,백승현,07-15,@롯데,W,9:3,9,3B,1,0,0,...,1,0,0.250,0.344,0.286,0.629,1,1.73,-0.23,-0.022


In [62]:
result_20_ff = pd.concat([result_20_f,extra_result_20_f])
result_20_ff

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,강현우,05-06,롯데,L,4:9,8,C,0,1,0,...,0,0,0.000,0.000,0.000,0.000,1,0.31,-0.12,-0.008
1,강현우,05-08,@두산,W,12:3,8,PR,0,0,0,...,0,0,0.000,0.000,0.000,0.000,0,0.00,0.00,0.000
2,강현우,05-13,@NC,L,4:5,8,C,0,0,0,...,0,0,0.500,0.500,1.000,1.500,0,0.00,0.00,0.000
3,강현우,05-16,삼성,W,10:4,2,C,0,0,0,...,0,0,0.500,0.500,1.000,1.500,0,0.00,0.00,0.000
4,강현우,05-21,한화,L,4:9,8,C,0,1,0,...,0,0,0.429,0.429,0.714,1.143,5,0.62,-0.46,-0.017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2618,백승현,07-07,@두산,L,6:9,8,3B,1,3,2,...,0,0,0.350,0.409,0.400,0.809,10,0.71,-0.05,-0.019
2619,백승현,07-09,@두산,L,0:6,8,3B,1,2,0,...,0,0,0.280,0.357,0.320,0.677,9,1.11,-0.52,-0.057
2620,백승현,07-11,NC,D,6:6,9,3B,1,2,0,...,2,0,0.250,0.344,0.286,0.629,15,0.92,-0.71,-0.037
2621,백승현,07-15,@롯데,W,9:3,9,3B,1,0,0,...,1,0,0.250,0.344,0.286,0.629,1,1.73,-0.23,-0.022


In [63]:
result_20_ff[result_20_ff.장타.isnull()]

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
6212,백승현,데이터가,없습니다.,검색,조건을,다시,확인하세요.,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [64]:
result_20_ff = result_20_ff.dropna(axis = 0)
result_20_ff

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,강현우,05-06,롯데,L,4:9,8,C,0,1,0,...,0,0,0.000,0.000,0.000,0.000,1,0.31,-0.12,-0.008
1,강현우,05-08,@두산,W,12:3,8,PR,0,0,0,...,0,0,0.000,0.000,0.000,0.000,0,0.00,0.00,0.000
2,강현우,05-13,@NC,L,4:5,8,C,0,0,0,...,0,0,0.500,0.500,1.000,1.500,0,0.00,0.00,0.000
3,강현우,05-16,삼성,W,10:4,2,C,0,0,0,...,0,0,0.500,0.500,1.000,1.500,0,0.00,0.00,0.000
4,강현우,05-21,한화,L,4:9,8,C,0,1,0,...,0,0,0.429,0.429,0.714,1.143,5,0.62,-0.46,-0.017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2618,백승현,07-07,@두산,L,6:9,8,3B,1,3,2,...,0,0,0.350,0.409,0.400,0.809,10,0.71,-0.05,-0.019
2619,백승현,07-09,@두산,L,0:6,8,3B,1,2,0,...,0,0,0.280,0.357,0.320,0.677,9,1.11,-0.52,-0.057
2620,백승현,07-11,NC,D,6:6,9,3B,1,2,0,...,2,0,0.250,0.344,0.286,0.629,15,0.92,-0.71,-0.037
2621,백승현,07-15,@롯데,W,9:3,9,3B,1,0,0,...,1,0,0.250,0.344,0.286,0.629,1,1.73,-0.23,-0.022


# 2.4) 2021년 선수정보

In [65]:
result_21 = []
manuallycheck_21 = []

url = 'http://www.statiz.co.kr/player.php?name=%s&search='

driver = webdriver.Chrome(executable_path = 'chromedriver')
bydateselect = 'body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(1) > div > div.col-xs-12.col-sm-8.col-md-8.col-lg-12 > div > div.table-responsive.no-padding.box > table > tbody > tr > td > a:nth-child(4)'
closead = '#foin_close_02eu > a > img'

for i in tqdm(range(len(player_list_21))):
    player_path = url % player_list_21[i]
    driver.implicitly_wait(3)    
    driver.get(player_path)
    
    try:
        select = driver.find_element_by_css_selector(bydateselect)
        select.click()
        driver.implicitly_wait(3)

        select2 = driver.find_element_by_css_selector(closead)
        select2.click()
        driver.implicitly_wait(3)    

        select3 = driver.find_element_by_partial_link_text("2021")
        driver.implicitly_wait(3)
        select3.click()
        driver.implicitly_wait(3)

        table = driver.find_element_by_class_name('table.table-striped.table-responsive.table-condensed.no-space.table-bordered')
        tbody = table.find_element_by_tag_name("tbody")
        rows = tbody.find_elements_by_class_name("oddrow_stz0")+tbody.find_elements_by_class_name("evenrow_stz0")

        for index, value in enumerate(rows):
            result_21.append([player_list_21[i],value.text])
        time.sleep(3)
    
    except NoSuchElementException:
        manuallycheck_21.append(player_list_21[i])

           
df_result_21 = pd.DataFrame(result_21)


result_split_21= []
for i in df_result_21[1]:
    result_split_21.append(i.split(' '))

result_21_f = pd.concat([df_result_21[0],pd.DataFrame(result_split_21)], axis = 1)
result_21_f.columns = ['이름', '날짜', '상대', '결과', '점수', '타순', 'P', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타', 'OPS', '투구', 'avLI', 'RE24', 'WPA']
result_21_f

100%|████████████████████████████████████████████████████████████████████████████████| 171/171 [28:14<00:00,  9.91s/it]


,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,라모스,04-04,@NC,W,2:1,2,1B,1,3,0,...,0,0,0.000,0.250,0.000,0.250,15,2.04,-1.12,-0.092
1,라모스,04-07,@KT,L,3:7,4,1B,1,2,0,...,0,0,0.111,0.200,0.111,0.311,6,1.85,-1.07,-0.093
2,라모스,04-10,SSG,L,3:4,2,1B,1,4,2,...,0,0,0.286,0.375,0.571,0.946,27,1.89,1.94,0.280
3,라모스,04-13,@키움,L,2:8,2,1B,1,4,0,...,0,0,0.190,0.261,0.381,0.642,18,0.50,-1.03,-0.052
4,라모스,04-15,@키움,W,6:4,4,1B,1,4,1,...,0,0,0.241,0.313,0.379,0.692,16,1.17,-1.04,-0.085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9260,안치홍,08-26,@KIA,D,3:3,6,2B,1,3,0,...,0,0,0.308,0.373,0.446,0.819,20,1.68,-1.89,-0.172
9261,안치홍,08-29,두산,W,4:2,6,2B,1,3,1,...,0,0,0.306,0.372,0.451,0.823,16,0.92,-0.45,-0.052
9262,안치홍,09-03,한화,W,6:3,6,2B,1,2,0,...,0,0,0.305,0.380,0.452,0.832,14,1.07,-1.05,-0.104
9263,안치홍,09-05,@NC,L,3:9,6,2B,1,4,1,...,0,0,0.305,0.377,0.459,0.836,19,0.37,1.50,-0.030


In [66]:
len(set(result_21_f.이름))

144

In [67]:
result_21_f[result_21_f.장타.isnull()]

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA


In [68]:
len(manuallycheck_21)

27

In [69]:
manuallycheck_21

['박주홍',
 '박정현',
 '이해창',
 '정진호',
 '이원재',
 '김민혁',
 '박찬호',
 '박준태',
 '김민수',
 '조용호',
 '김태훈',
 '김재성',
 '최원준',
 '이진영',
 '박준영',
 '김민수',
 '이정훈',
 '페르난데스',
 '박병호',
 '이원석',
 '이병규',
 '김현수',
 '김성현',
 '이재원',
 '이명기',
 '박건우',
 '김상수']

In [70]:
hitter_player_21[hitter_player_21.NAME == '김상수']

,GYEAR,PCODE,GAMENUM,PA,AB,BA,HIT,HR,TOTB,SLG,...,BB,KK,IB,HP,GD,NAME,T_ID,POSITION,AGE_VA,MONEY
243,2021,79402,70,278,237,0.211,50,1,61,0.257,...,30,26,0,3,7,김상수,SS,내,31,25000만원


In [71]:
#백창수(백진우), 고장혁, 나종덕(나균안) 검색 결과 없음

extra_21 = [
'http://www.statiz.co.kr/player.php?opt=3&name=%EB%B0%95%EC%A3%BC%ED%99%8D&birth=2001-04-16', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EB%B0%95%EC%A0%95%ED%98%84&birth=2001-07-27', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%9D%B4%ED%95%B4%EC%B0%BD&birth=1987-05-11', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%A0%95%EC%A7%84%ED%98%B8&birth=1988-10-02', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%9D%B4%EC%9B%90%EC%9E%AC&birth=1989-05-20', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EA%B9%80%EB%AF%BC%ED%98%81&birth=1995-11-21', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EB%B0%95%EC%B0%AC%ED%98%B8&birth=1995-06-05', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EB%B0%95%EC%A4%80%ED%83%9C&birth=1991-07-26', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EA%B9%80%EB%AF%BC%EC%88%98&birth=1991-03-02', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EA%B9%80%EB%AF%BC%EC%88%98&birth=1998-03-18', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%A1%B0%EC%9A%A9%ED%98%B8&birth=1989-09-09', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EA%B9%80%ED%83%9C%ED%9B%88&birth=1996-03-31', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EA%B9%80%EC%9E%AC%EC%84%B1&birth=1996-10-30', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%B5%9C%EC%9B%90%EC%A4%80&birth=1997-03-23', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%9D%B4%EC%A7%84%EC%98%81&birth=1997-07-21', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EB%B0%95%EC%A4%80%EC%98%81&birth=1997-08-05', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%9D%B4%EC%A0%95%ED%9B%88&birth=1994-12-07', 
'http://www.statiz.co.kr/player.php?opt=3&name=%ED%8E%98%EB%A5%B4%EB%82%9C%EB%8D%B0%EC%8A%A4&birth=1988-04-27', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EB%B0%95%EB%B3%91%ED%98%B8&birth=1986-07-10', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%9D%B4%EC%9B%90%EC%84%9D&birth=1986-10-21', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%9D%B4%EB%B3%91%EA%B7%9C&birth=1983-10-09', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EA%B9%80%ED%98%84%EC%88%98&birth=1988-01-12', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EA%B9%80%EC%84%B1%ED%98%84&birth=1987-03-09', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%9D%B4%EC%9E%AC%EC%9B%90&birth=1988-02-24', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EC%9D%B4%EB%AA%85%EA%B8%B0&birth=1987-12-26', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EB%B0%95%EA%B1%B4%EC%9A%B0&birth=1990-09-08', 
'http://www.statiz.co.kr/player.php?opt=3&name=%EA%B9%80%EC%83%81%EC%88%98&birth=1990-03-23'] 

extra_21_name = ['박주홍','박정현','이해창', '정진호','이원재','김민혁','박찬호','박준태','SS김민수','LT김민수','조용호','김태훈',
                 '김재성','최원준','이진영','박준영','이정훈','페르난데스','박병호','이원석','이병규','김현수','김성현','이재원',
                 '이명기','박건우','김상수']

In [72]:
len(extra_21), len(extra_21_name)

(27, 27)

In [73]:
extra_result_21 = []


driver = webdriver.Chrome(executable_path = 'chromedriver')
bydateselect = 'body > div.wrapper > div.content-wrapper > div > section.content > div > div:nth-child(1) > div > div.col-xs-12.col-sm-8.col-md-8.col-lg-12 > div > div.table-responsive.no-padding.box > table > tbody > tr > td > a:nth-child(4)'
closead = '#foin_close_02eu > a > img'

for i in tqdm(range(len(extra_21))):
    player_path = i
    driver.implicitly_wait(3)    
    driver.get(extra_21[i])
    
    table = driver.find_element_by_class_name('table.table-striped.table-responsive.table-condensed.no-space.table-bordered')
    tbody = table.find_element_by_tag_name("tbody")
    rows = tbody.find_elements_by_class_name("oddrow_stz0")+tbody.find_elements_by_class_name("evenrow_stz0")

    for index, value in enumerate(rows):
        extra_result_21.append([extra_21_name[i],value.text])
    time.sleep(3)
    
           
df_extra_21 = pd.DataFrame(extra_result_21)


extra_result_21_split= []
for i in df_extra_21[1]:
    extra_result_21_split.append(i.split(' '))

extra_result_21_f = pd.concat([df_extra_21[0],pd.DataFrame(extra_result_21_split)], axis = 1)
extra_result_21_f.columns = ['이름', '날짜', '상대', '결과', '점수', '타순', 'P', '선발', '타수', '득점', '안타', '2타', '3타', '홈런', '루타', '타점', '도루', '도실', '볼넷', '사구', '고4', '삼진', '병살', '희타', '희비', '타율', '출루', '장타', 'OPS', '투구', 'avLI', 'RE24', 'WPA']
extra_result_21_f

100%|██████████████████████████████████████████████████████████████████████████████████| 27/27 [03:08<00:00,  6.99s/it]


,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,박주홍,05-08,@SSG,D,:,7,DH,1,5,1,...,0,0,0.200,0.200,0.400,0.600,21,0.26,-0.69,0.030
1,박주홍,05-09,@SSG,D,:,6,DH,1,3,0,...,0,0,0.111,0.273,0.222,0.495,13,1.51,-1.27,-0.126
2,박주홍,05-12,@두산,W,3:0,8,DH,1,3,0,...,0,0,0.143,0.333,0.214,0.548,12,0.75,-0.38,-0.036
3,박주홍,05-15,한화,W,15:1,6,LF,0,0,0,...,0,0,0.176,0.364,0.235,0.599,0,0.00,0.00,0.000
4,박주홍,05-19,@삼성,W,9:2,9,PH,0,1,0,...,0,0,0.211,0.375,0.316,0.691,1,0.03,0.23,0.002
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,김상수,08-29,@KT,L,3:8,9,2B,1,3,0,...,0,0,0.223,0.312,0.265,0.577,18,0.50,0.66,0.046
1566,김상수,09-01,키움,W,7:1,9,2B,1,2,2,...,0,0,0.227,0.320,0.268,0.588,18,0.67,2.07,0.098
1567,김상수,09-03,@KIA,W,9:3,6,2B,1,3,1,...,0,0,0.228,0.322,0.268,0.590,11,0.57,0.90,0.060
1568,김상수,09-05,두산,L,5:6,9,2B,1,3,0,...,0,0,0.228,0.324,0.267,0.591,15,1.78,-0.02,-0.057


In [74]:
result_21_ff = pd.concat([result_21_f,extra_result_21_f])
result_21_ff

,이름,날짜,상대,결과,점수,타순,P,선발,타수,득점,...,희타,희비,타율,출루,장타,OPS,투구,avLI,RE24,WPA
0,라모스,04-04,@NC,W,2:1,2,1B,1,3,0,...,0,0,0.000,0.250,0.000,0.250,15,2.04,-1.12,-0.092
1,라모스,04-07,@KT,L,3:7,4,1B,1,2,0,...,0,0,0.111,0.200,0.111,0.311,6,1.85,-1.07,-0.093
2,라모스,04-10,SSG,L,3:4,2,1B,1,4,2,...,0,0,0.286,0.375,0.571,0.946,27,1.89,1.94,0.280
3,라모스,04-13,@키움,L,2:8,2,1B,1,4,0,...,0,0,0.190,0.261,0.381,0.642,18,0.50,-1.03,-0.052
4,라모스,04-15,@키움,W,6:4,4,1B,1,4,1,...,0,0,0.241,0.313,0.379,0.692,16,1.17,-1.04,-0.085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1565,김상수,08-29,@KT,L,3:8,9,2B,1,3,0,...,0,0,0.223,0.312,0.265,0.577,18,0.50,0.66,0.046
1566,김상수,09-01,키움,W,7:1,9,2B,1,2,2,...,0,0,0.227,0.320,0.268,0.588,18,0.67,2.07,0.098
1567,김상수,09-03,@KIA,W,9:3,6,2B,1,3,1,...,0,0,0.228,0.322,0.268,0.590,11,0.57,0.90,0.060
1568,김상수,09-05,두산,L,5:6,9,2B,1,3,0,...,0,0,0.228,0.324,0.267,0.591,15,1.78,-0.02,-0.057


# 3. 최종데이터

In [75]:
result_18_ff.set_index('이름', inplace = True)
result_19_ff.set_index('이름', inplace = True)
result_20_ff.set_index('이름', inplace = True)
result_21_ff.set_index('이름', inplace = True)

In [76]:
result_18_ff.to_csv('선수정보_2018.csv')
result_19_ff.to_csv('선수정보_2019.csv')
result_20_ff.to_csv('선수정보_2020.csv')
result_21_ff.to_csv('선수정보_2021.csv')